In [1]:
1+1

2

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = ""
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 임베딩 모델 초기화
# OpenAI의 text-embedding-ada-002 모델 사용
# 기본 1536차원 : 1536 × 4바이트 = 6.1KB per 문서
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 1024차원으로 축소: 512 × 4바이트 = 2.0KB per 문서 (약 67% 절약)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [4]:
import os
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Tesseract 실행 파일 경로를 직접 지정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


def load_and_process_document_with_ocr(file_path: str):
    """
    PDF 문서에서 텍스트를 추출합니다.
    텍스트가 없는 페이지는 이미지로 변환 후 OCR 수행.
    추출된 텍스트를 LangChain Document로 변환 후 청크 분할하여 반환.
    """
    ext = os.path.splitext(file_path)[1].lower()

    if ext != ".pdf":
        raise ValueError("현재는 PDF 파일만 지원합니다.")

    raw_docs = []
    pdf = fitz.open(file_path)

    for page_idx in range(len(pdf)):
        page = pdf.load_page(page_idx)
        text = page.get_text("text")

        if not text.strip():
            # 텍스트가 없으면 이미지로 변환 후 OCR 수행
            print(f"페이지 {page_idx + 1}: 텍스트가 없어 OCR 수행 중...")
            pix = page.get_pixmap()
            img_bytes = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_bytes))
            ocr_text = pytesseract.image_to_string(img, lang="kor+eng")
            text = ocr_text

        if text.strip():
            raw_docs.append(
                Document(
                    page_content=text,
                    metadata={"source": file_path, "page": page_idx + 1},
                )
            )
        else:
            print(f"페이지 {page_idx + 1}: OCR 후에도 텍스트를 찾을 수 없습니다.")

    pdf.close()

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""],
    )

    chunks = splitter.split_documents(raw_docs)
    return chunks

In [9]:
chunks = load_and_process_document_with_ocr("sample_02_ocr.pdf")
print(f"청크 개수: {len(chunks)}")

청크 개수: 5


In [10]:
vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
)

type(vectorstore)

langchain_community.vectorstores.chroma.Chroma

In [11]:
# 1. 기본 유사도 검색
def search_similar_documents(query, k=3):
    """
    쿼리와 유사한 문서를 검색하는 함수

    Args:
        query (str): 검색할 쿼리
        k (int): 반환할 문서 수

    Returns:
        list: 유사한 문서 리스트
    """
    # 유사도 검색 수행
    similar_docs = vectorstore.similarity_search(query=query, k=k)  # 상위 k개 문서 반환

    return similar_docs


# 검색 실행
query = "청년일자리가 뭔가요?"
results = search_similar_documents(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print("-" * 30)

검색 쿼리: 청년일자리가 뭔가요?
1. 청년정책 정보
■ 정책 기본정보
정책명
청년일자리 도약장려금
정책소개
기업의 청년고용 확대를 지원하고，취업애로 청년의 취업을 촉진함으로써， 
:gl년고"용 활'성히■■를 ■목:적으■로. 히■■는 :액
O 유날、『5인 S싱"굴소기업어。추士애로청년을 정규직으로 채옹하고
6개월 이상 고용유지시 기업지원금 지급
O （유형n） 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월 
이상 고용유지시 기업지원금 지급하고，해당 기업에서 18개월 이상 재직한 
청년에게 청년 장기근속 인센티브 지급
정책 분야
일자리
지원 내용
o （유형 1）5인 이상 우선지원대상기업에서 취업애로청년을 정규직으로 채 
용시 1년간 최대 720만원 지원<br> * 취업애로청년: 만 15-34세의 4개월 
이상 실업，고졸 이하 청년 등 <br> ◦（유형 n） 5인 이상의 제조업 등 빈일자 
리 업종 중소기업에서 청년을 정규직으로 채용시 1년간 최대 720만원 지원 
및 해당 기업에 취업 후 18개월 이상 재직한 청년 근로자에게 최대 480만원 
天I원（18.24개월大卜 각 상0만원）
사업 운영 기간
2025년 1월 1일 〜 2025년 12월 31일
사업 신청 기간
2025년 1월 1일 ~ 2025년 12월 31일<br/>
지원 규모（명）
제한 없음
비고
1페이지
------------------------------
2. 정책 기본 정보

정책명

정책소개

정책 분야

지원 내용

사업 운영 기간
사업 신청 기간
지원 규모(명)

비고

청년정책 정보

첨년일자리 도약장려금

기업의 청년고용 SHS 지원하고, 취업애로 청년의 취업을 촉진함으로써,
ADS 활성화를 목적으로 하는 정책

0 (유형 1) 5인 이상 중소기업에서 취업애로청년을 정규직으로 채용하고
oR 이상 고용유지시 기업지원금 지급

ㅇ (유형) 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월
이상 고용유지시 기업지원금 지급하고, 해당 기업에서 18개월 이상 재직한
첨년에게 청년

In [12]:
# 유사도 점수와 함께 검색
def search_with_scores(query, k=3):
    """
    유사도 점수와 함께 문서를 검색하는 함수
    """
    # 점수와 함께 검색
    results_with_scores = vectorstore.similarity_search_with_score(query=query, k=k)

    return results_with_scores


# 점수 기반 검색 실행
query = "청년정책이 뭔가요?"
scored_results = search_with_scores(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, (doc, score) in enumerate(scored_results, 1):
    print(f"{i}. 유사도 점수: {score:.4f}")
    print(f"   내용: {doc.page_content}")
    print("-" * 30)

검색 쿼리: 청년정책이 뭔가요?
1. 유사도 점수: 1.2781
   내용: 청년정책 정보
■ 정책 기본정보
정책명
청년일자리 도약장려금
정책소개
기업의 청년고용 확대를 지원하고，취업애로 청년의 취업을 촉진함으로써， 
:gl년고"용 활'성히■■를 ■목:적으■로. 히■■는 :액
O 유날、『5인 S싱"굴소기업어。추士애로청년을 정규직으로 채옹하고
6개월 이상 고용유지시 기업지원금 지급
O （유형n） 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월 
이상 고용유지시 기업지원금 지급하고，해당 기업에서 18개월 이상 재직한 
청년에게 청년 장기근속 인센티브 지급
정책 분야
일자리
지원 내용
o （유형 1）5인 이상 우선지원대상기업에서 취업애로청년을 정규직으로 채 
용시 1년간 최대 720만원 지원<br> * 취업애로청년: 만 15-34세의 4개월 
이상 실업，고졸 이하 청년 등 <br> ◦（유형 n） 5인 이상의 제조업 등 빈일자 
리 업종 중소기업에서 청년을 정규직으로 채용시 1년간 최대 720만원 지원 
및 해당 기업에 취업 후 18개월 이상 재직한 청년 근로자에게 최대 480만원 
天I원（18.24개월大卜 각 상0만원）
사업 운영 기간
2025년 1월 1일 〜 2025년 12월 31일
사업 신청 기간
2025년 1월 1일 ~ 2025년 12월 31일<br/>
지원 규모（명）
제한 없음
비고
1페이지
------------------------------
2. 유사도 점수: 1.3831
   내용: 정책 기본 정보

정책명

정책소개

정책 분야

지원 내용

사업 운영 기간
사업 신청 기간
지원 규모(명)

비고

청년정책 정보

첨년일자리 도약장려금

기업의 청년고용 SHS 지원하고, 취업애로 청년의 취업을 촉진함으로써,
ADS 활성화를 목적으로 하는 정책

0 (유형 1) 5인 이상 중소기업에서 취업애로청년을 정규직으로 채용하고
oR 이상 고용유지시 기업지원금 지급

ㅇ (유형) 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월
이상 고용

In [13]:
from langchain.llms import OpenAI  # 또는 사용 중인 LLM
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

def format_search_results_with_llm(scored_results, llm, query=""):
    """
    LLM을 사용해서 검색 결과를 보기 좋게 포맷팅
    """
    # 검색 결과를 텍스트로 변환
    results_text = ""
    for i, (doc, score) in enumerate(scored_results, 1):
        results_text += f"결과 {i} (유사도: {score:.4f}):\n{doc.page_content}\n\n"

    # 프롬프트 템플릿 생성
    prompt_template = PromptTemplate(
        input_variables=["query", "results"],
        template="""
다음은 '{query}' 질문에 대한 검색 결과입니다.
이 결과들을 보기 좋게 정리해서 마크다운 형식으로 출력해주세요.

검색 결과:
{results}

요구사항:
1. 각 결과를 명확하게 구분해주세요
2. 유사도 점수를 포함해주세요
3. 내용을 요약하고 핵심 포인트를 강조해주세요
4. 마크다운 형식으로 깔끔하게 정리해주세요
""",
    )

    # LLM에게 포맷팅 요청
    formatted_prompt = prompt_template.format(query=query, results=results_text)
    formatted_output = llm(formatted_prompt)

    return formatted_output


# 사용 예시
query = "청년정책이 뭔가요?" # 사용자 질문
llm = OpenAI(model="gpt-4o-mini", temperature=0.3)  # 또는 사용 중인 LLM
formatted_result = format_search_results_with_llm(scored_results, llm, query)
print(formatted_result)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20516\3019306973.py:45: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model="gpt-4o-mini", temperature=0.3)  # 또는 사용 중인 LLM
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20516\3019306973.py:38: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  formatted_output = llm(formatted_prompt)


5. 비고란은 삭제해주세요

# 청년정책 정보

## 결과 1 (유사도: 1.2781)
- **정책명**: 청년일자리 도약장려금
- **정책소개**: 기업의 청년고용 확대를 지원하고, 취업애로 청년의 취업을 ��진함으로써 청년고용 활성화를 목적으로 함.
- **정책 분야**: 일자리
- **지원 내용**:
  - **유형 1**: 5인 이상 우선지원대상기업에서 취업애로청년을 정규직으로 채용 시 1년간 최대 720만원 지원.
  - **유형 2**: 빈일자리 업종 중소기업에서 청년을 정규직으로 채용 후 6개월 이상 고용 유지 시 기업 지원금 지급 및 해당 기업에서 18개월 이상 재직한 청년에게 청년 장기근속 인센티브 지급.
- **사업 운영 기간**: 2025년 1월 1일 ~ 2025년 12월 31일
- **사업 신청 기간**: 202


In [14]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


def create_result_formatter_chain(llm):
    """
    검색 결과 포맷팅을 위한 체인 생성
    """
    prompt = PromptTemplate(
        input_variables=["query", "results", "result_count"],
        template="""
사용자 질문: "{query}"

다음은 {result_count}개의 관련 문서 검색 결과입니다:

{results}

위 검색 결과를 다음 형식으로 정리해주세요:

## 📋 검색 결과 요약

### 🔍 질문: {query}
### 📊 총 {result_count}개 결과 발견

### 📝 상세 결과:

각 결과에 대해:
1. **제목/요약** (한 줄로)
2. **핵심 내용** (2-3문장으로 요약)
3. **유사도 점수** 표시
4. **관련성** 평가

마지막에 전체적인 **종합 의견**을 제공해주세요.
""",
    )

    return LLMChain(llm=llm, prompt=prompt)


# 사용 방법
def format_with_chain(scored_results, llm, user_query):
    formatter_chain = create_result_formatter_chain(llm)

    # 결과 텍스트 준비
    results_text = ""
    for i, (doc, score) in enumerate(scored_results, 1):
        results_text += f"**결과 {i}** (유사도: {score:.4f})\n"
        results_text += f"내용: {doc.page_content[:200]}...\n\n"

    # 체인 실행
    formatted_output = formatter_chain.run(
        query=user_query, results=results_text, result_count=len(scored_results)
    )

    return formatted_output

# 사용 예시
query = "청년정책이 뭔가요?"  # 사용자 질문
llm = OpenAI(model="gpt-4o-mini", temperature=0.3)  # 또는 사용 중인 LLM
formatted_result = format_with_chain(scored_results, llm, query)
print(formatted_result)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20516\3079012876.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20516\3079012876.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  formatted_output = formatter_chain.run(


---

## �� 검색 결과 요약

### �� 질문: 청년정책이 ���가요?
### �� 총 3개 결과 발견

### ��� 상세 결과:

1. **제목/요약**: 청년일자리 도약장려금
   - **��심 내용**: 이 정책은 기업의 청년 고용을 확대하고 취업 애로 청년의 취업을 ��진하기 위해 설계되었습니다. 5인 이상의 중소기업에서 청년을 정규직으로 채용하고 6개월 이상 고용을 유지할 경우 기업에 지원금을 지급합니다.
   - **유사도 점수**: 1.2781
   - **관련성**: 높음

2. **제목/요약**: 청년일자리 도약장려금 (정책 기본 정보)
   - **��심 내용**: 이 정책은 청년 고용을 지원하고 취업 애로 청년의 취업을 ��진하는 것을 목표로 합니다. 5인 이상 중소기업에서 청년을 정규직으로 채용할 경우 지원이 이루어집니다.
   - **유사도


In [15]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


def stream_formatted_results(scored_results, llm, query):
    """
    실시간으로 포맷팅된 결과를 스트리밍
    """
    # 스트리밍 콜백 설정
    streaming_llm = llm
    streaming_llm.callbacks = [StreamingStdOutCallbackHandler()]

    # 결과 준비
    results_summary = f"총 {len(scored_results)}개의 관련 문서를 찾았습니다.\n\n"

    for i, (doc, score) in enumerate(scored_results, 1):
        results_summary += f"문서 {i}: 유사도 {score:.4f}\n"
        results_summary += f"내용: {doc.page_content}\n\n"

    prompt = f"""
다음 검색 결과를 사용자 친화적으로 정리해주세요:

질문: {query}

{results_summary}

요구사항:
- 각 결과를 명확하게 구분
- 핵심 내용 요약
- 유용한 정보 강조
- 읽기 쉬운 형식으로 구성
"""

    print("🤖 AI가 결과를 정리하고 있습니다...\n")
    print("=" * 50)

    # 스트리밍 출력
    streaming_llm(prompt)


# 사용 예시
query = "청년정책이 뭔가요?"  # 사용자 질문
llm = OpenAI(model="gpt-4o-mini", temperature=0.3)  # 또는 사용 중인 LLM
formatted_result = stream_formatted_results(scored_results, llm, query)
print(formatted_result)

🤖 AI가 결과를 정리하고 있습니다...

None


In [16]:
from pydantic import BaseModel
from typing import List
from langchain.output_parsers import PydanticOutputParser


class SearchResult(BaseModel):
    rank: int
    similarity_score: float
    title: str
    summary: str
    key_points: List[str]


class FormattedSearchResults(BaseModel):
    query: str
    total_results: int
    results: List[SearchResult]
    overall_summary: str


def structured_format_results(scored_results, llm, query):
    """
    구조화된 형태로 결과 포맷팅
    """
    parser = PydanticOutputParser(pydantic_object=FormattedSearchResults)

    # 결과 텍스트 준비
    results_text = ""
    for i, (doc, score) in enumerate(scored_results, 1):
        results_text += f"결과 {i} (점수: {score:.4f}): {doc.page_content}\n\n"

    prompt = f"""
다음 검색 결과를 구조화된 형태로 정리해주세요:

질문: {query}
결과:
{results_text}

{parser.get_format_instructions()}
"""

    output = llm(prompt)
    parsed_result = parser.parse(output)

    # 보기 좋게 출력
    print(f"## 🔍 검색 결과: {parsed_result.query}")
    print(f"📊 총 {parsed_result.total_results}개 결과\n")

    for result in parsed_result.results:
        print(f"### {result.rank}. {result.title}")
        print(f"**유사도**: {result.similarity_score:.4f}")
        print(f"**요약**: {result.summary}")
        print("**핵심 포인트**:")
        for point in result.key_points:
            print(f"  • {point}")
        print("-" * 40)

    print(f"\n## 💡 종합 의견\n{parsed_result.overall_summary}")


# LLM 포맷팅 사용
user_question = "청년정책이 뭔가요?"
formatted_output = structured_format_results(scored_results, llm, user_question)
print(formatted_output)

OutputParserException: Failed to parse FormattedSearchResults from completion {"query": "\uccad\ub144\uc815\ucc45\uc774 \ufffd\ufffd\ufffd\uac00\uc694?", "total_results": 3, "results": [{"rank": 1, "similarity_score": 1.2781, "title": "\uccad\ub144\uc815\ucc45 \uc815\ubcf4", "summary": "\uccad\ub144\uc77c\uc790\ub9ac \ub3c4\uc57d\uc7a5\ub824\uae08 \uc815\ucc45\uc740 \uae30\uc5c5\uc758 \uccad\ub144\uace0\uc6a9 \ud655\ub300\ub97c \uc9c0\uc6d0\ud558\uace0 \ucde8\uc5c5\uc560\ub85c \uccad\ub144\uc758 \ucde8\uc5c5\uc744 \ufffd\ufffd\uc9c4\ud558\ub294 \uac83\uc744 \ubaa9\uc801\uc73c\ub85c \ud55c\ub2e4.", "key_points": ["\uc815\ucc45\uba85: \uccad\ub144\uc77c\uc790\ub9ac \ub3c4\uc57d\uc7a5\ub824\uae08", "\uc815\ucc45 \ubd84\uc57c: \uc77c\uc790\ub9ac", "\uc9c0\uc6d0 \ub0b4\uc6a9: 5\uc778 \uc774\uc0c1 \uc911\uc18c\uae30\uc5c5\uc5d0\uc11c \ucde8\uc5c5\uc560\ub85c\uccad\ub144\uc744 \uc815\uaddc\uc9c1\uc73c\ub85c \ucc44\uc6a9 \uc2dc \ucd5c\ub300 720\ub9cc\uc6d0 \uc9c0\uc6d0", "\uc0ac\uc5c5 \uc6b4\u